In [1]:
import GSDUncertainty
import GrainSizing

In [2]:
PATH = "F:/cellpose/code_demo/"
gsds = GrainSizing.load.load_grain_set([PATH],gsd_str='re_scaled',filter_str='0')

In [3]:
res_dict_bs = GSDUncertainty.calculate.dataset_uncertainty(gsds,mute=True)

bootstrapping: 100%|██████████| 7/7 [00:01<00:00,  6.85gsd/s]


In [4]:
res_dict_mc = GSDUncertainty.calculate.dataset_uncertainty(gsds,method='MC',scale_err=[.1],length_err=[1],num_it=100,mute=True)

MC: 100%|██████████| 7/7 [01:05<00:00,  9.36s/gsd]


In [5]:
sfm_error_l = GSDUncertainty.calculate.compile_sfm_error(from_file=PATH+'OM_err.csv')

In [6]:
res_dict_om = GSDUncertainty.calculate.dataset_uncertainty(gsds,method='MC_SfM',sfm_error=sfm_error_l,sfm_type='OM',num_it=100,mute=True)

MC_SfM: 100%|██████████| 7/7 [01:05<00:00,  9.42s/gsd]


In [7]:
res_dict_si = GSDUncertainty.calculate.dataset_uncertainty(gsds,method='MC_SfM',sfm_error=sfm_error_l,num_it=100,mute=True)

MC_SfM: 100%|██████████| 7/7 [01:09<00:00,  9.98s/gsd]
